Nếu **`page_content`** chính là **answer** trong trường hợp của bạn, thì bạn có thể đồng nhất việc sử dụng trường này làm câu trả lời trực tiếp khi tìm thấy tài liệu có độ tương đồng cao. Dưới đây là cách tối ưu pipeline để xử lý kịch bản này.

---

### Cách xử lý tối ưu:

#### 1. **Cấu trúc dữ liệu trong Qdrant**
Mỗi tài liệu trong Qdrant cần có:
- `vector`: Biểu diễn vector hóa của nội dung tài liệu hoặc câu hỏi liên quan.
- `payload`: 
  - `page_content`: Nội dung câu trả lời hoặc thông tin cần thiết.
  - `metadata`: Thông tin bổ sung (như nguồn tài liệu, câu hỏi liên quan, v.v.).

Ví dụ:
```json
{
    "id": "1",
    "vector": [0.123, 0.234, 0.345, ...],
    "payload": {
        "page_content": "Luật này quy định về quy tắc giao thông đường bộ...",
        "metadata": {
            "source": "LegalRAG.xlsx",
            "question": "Trình bày khái niệm, chế độ pháp lý vùng nội thủy..."
        }
    }
}
```

---

#### 2. **Hàm `pipe` được tối ưu hóa**
Khi tìm thấy tài liệu có **`page_content`** với độ tương đồng cao, trả về nội dung ngay lập tức.

```python
def pipe(self, user_message: str, model_id: str, messages: List[dict], body: dict) -> Union[str, Generator, Iterator]:
    """Process user message and return relevant context"""
    try:
        print(f"Processing: {user_message}")

        # Tạo embedding và tìm kiếm câu hỏi tương tự trong Qdrant
        query_vector = self.embeddings.embed_query(user_message)
        results = self.search_vectors(query_vector)

        if "error" in results:
            return f"Search error: {results['error']}"

        matches = results.get("result", [])
        if matches:
            for match in matches:
                score = float(match.get("score", 0))
                payload = match.get("payload", {})

                # Trả về page_content trực tiếp nếu độ tương đồng >= 0.8
                if score >= 0.8 and "page_content" in payload:
                    return f"Câu trả lời từ tài liệu: {payload['page_content']} (Nguồn: {payload.get('metadata', {}).get('source', 'Không rõ')})"

        # Nếu không tìm thấy, tạo câu trả lời qua OpenAI
        print("Không tìm thấy tài liệu phù hợp, tạo câu trả lời qua OpenAI...")
        context = [
            match.get("payload", {}).get("page_content", "No content")
            for match in matches if match.get("score", 0) > 0.5
        ]

        messages = [
            {
                "role": "system",
                "content": (
                    "Bạn là trợ lý AI giúp trả lời các câu hỏi dựa trên tài liệu pháp lý. "
                    "Dưới đây là ngữ cảnh:\n\n" + "\n\n".join(context)
                )
            },
            {
                "role": "user",
                "content": user_message
            }
        ]

        response = asyncio.run(self.get_completion(messages))
        return response

    except Exception as e:
        print(f"Pipeline error: {str(e)}")
        return f"Error: {str(e)}"
```

---

#### 3. **Cập nhật hàm `search_vectors`**
Tối ưu hàm tìm kiếm để đảm bảo `page_content` và `metadata` được trả về.

```python
def search_vectors(self, query_vector: List[float], top_k: int = 5) -> dict:
    """Search Qdrant collection"""
    try:
        payload = {
            "vector": query_vector,
            "limit": top_k,
            "with_payload": True,  # Trả về payload chứa page_content và metadata
            "score_threshold": 0.8
        }

        url = f"{self.valves.QDRANT_API_URL}/collections/{self.valves.QDRANT_COLLECTION}/points/search"
        headers = {
            "Authorization": f"Bearer {self.valves.QDRANT_API_KEY}",
            "Content-Type": "application/json; charset=utf-8",
        }

        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        return {"result": response.json().get("result", [])}
    except Exception as e:
        print(f"Search error: {str(e)}")
        return {"error": str(e)}
```

---

#### 4. **Kết quả đầu ra**
- **Nếu tìm thấy câu hỏi tương đồng:** 
  ```plaintext
  Câu trả lời từ tài liệu: Luật này quy định về quy tắc giao thông đường bộ... (Nguồn: LegalRAG.xlsx)
  ```
- **Nếu không tìm thấy:** 
  OpenAI sẽ được gọi để tạo câu trả lời dựa trên ngữ cảnh.

---

#### 5. **Tùy chọn mở rộng**
- Bạn có thể bổ sung logic để trả về cả **câu hỏi gốc trong metadata** nhằm tăng tính trực quan:
  ```python
  return f"Câu trả lời từ tài liệu: {payload['page_content']} (Nguồn: {payload.get('metadata', {}).get('source', 'Không rõ')}, Câu hỏi gốc: {payload.get('metadata', {}).get('question', 'Không rõ')})"
  ```

Nếu **`page_content`** chính là **answer** trong trường hợp của bạn, thì bạn có thể đồng nhất việc sử dụng trường này làm câu trả lời trực tiếp khi tìm thấy tài liệu có độ tương đồng cao. Dưới đây là cách tối ưu pipeline để xử lý kịch bản này.

---

### Cách xử lý tối ưu:

#### 1. **Cấu trúc dữ liệu trong Qdrant**
Mỗi tài liệu trong Qdrant cần có:
- `vector`: Biểu diễn vector hóa của nội dung tài liệu hoặc câu hỏi liên quan.
- `payload`: 
  - `page_content`: Nội dung câu trả lời hoặc thông tin cần thiết.
  - `metadata`: Thông tin bổ sung (như nguồn tài liệu, câu hỏi liên quan, v.v.).

Ví dụ:
```json
{
    "id": "1",
    "vector": [0.123, 0.234, 0.345, ...],
    "payload": {
        "page_content": "Luật này quy định về quy tắc giao thông đường bộ...",
        "metadata": {
            "source": "LegalRAG.xlsx",
            "question": "Trình bày khái niệm, chế độ pháp lý vùng nội thủy..."
        }
    }
}
```

---

#### 2. **Hàm `pipe` được tối ưu hóa**
Khi tìm thấy tài liệu có **`page_content`** với độ tương đồng cao, trả về nội dung ngay lập tức.

```python
def pipe(self, user_message: str, model_id: str, messages: List[dict], body: dict) -> Union[str, Generator, Iterator]:
    """Process user message and return relevant context"""
    try:
        print(f"Processing: {user_message}")

        # Tạo embedding và tìm kiếm câu hỏi tương tự trong Qdrant
        query_vector = self.embeddings.embed_query(user_message)
        results = self.search_vectors(query_vector)

        if "error" in results:
            return f"Search error: {results['error']}"

        matches = results.get("result", [])
        if matches:
            for match in matches:
                score = float(match.get("score", 0))
                payload = match.get("payload", {})

                # Trả về page_content trực tiếp nếu độ tương đồng >= 0.8
                if score >= 0.8 and "page_content" in payload:
                    return f"Câu trả lời từ tài liệu: {payload['page_content']} (Nguồn: {payload.get('metadata', {}).get('source', 'Không rõ')})"

        # Nếu không tìm thấy, tạo câu trả lời qua OpenAI
        print("Không tìm thấy tài liệu phù hợp, tạo câu trả lời qua OpenAI...")
        context = [
            match.get("payload", {}).get("page_content", "No content")
            for match in matches if match.get("score", 0) > 0.5
        ]

        messages = [
            {
                "role": "system",
                "content": (
                    "Bạn là trợ lý AI giúp trả lời các câu hỏi dựa trên tài liệu pháp lý. "
                    "Dưới đây là ngữ cảnh:\n\n" + "\n\n".join(context)
                )
            },
            {
                "role": "user",
                "content": user_message
            }
        ]

        response = asyncio.run(self.get_completion(messages))
        return response

    except Exception as e:
        print(f"Pipeline error: {str(e)}")
        return f"Error: {str(e)}"
```

---

#### 3. **Cập nhật hàm `search_vectors`**
Tối ưu hàm tìm kiếm để đảm bảo `page_content` và `metadata` được trả về.

```python
def search_vectors(self, query_vector: List[float], top_k: int = 5) -> dict:
    """Search Qdrant collection"""
    try:
        payload = {
            "vector": query_vector,
            "limit": top_k,
            "with_payload": True,  # Trả về payload chứa page_content và metadata
            "score_threshold": 0.8
        }

        url = f"{self.valves.QDRANT_API_URL}/collections/{self.valves.QDRANT_COLLECTION}/points/search"
        headers = {
            "Authorization": f"Bearer {self.valves.QDRANT_API_KEY}",
            "Content-Type": "application/json; charset=utf-8",
        }

        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        return {"result": response.json().get("result", [])}
    except Exception as e:
        print(f"Search error: {str(e)}")
        return {"error": str(e)}
```

---

#### 4. **Kết quả đầu ra**
- **Nếu tìm thấy câu hỏi tương đồng:** 
  ```plaintext
  Câu trả lời từ tài liệu: Luật này quy định về quy tắc giao thông đường bộ... (Nguồn: LegalRAG.xlsx)
  ```
- **Nếu không tìm thấy:** 
  OpenAI sẽ được gọi để tạo câu trả lời dựa trên ngữ cảnh.

---

#### 5. **Tùy chọn mở rộng**
- Bạn có thể bổ sung logic để trả về cả **câu hỏi gốc trong metadata** nhằm tăng tính trực quan:
  ```python
  return f"Câu trả lời từ tài liệu: {payload['page_content']} (Nguồn: {payload.get('metadata', {}).get('source', 'Không rõ')}, Câu hỏi gốc: {payload.get('metadata', {}).get('question', 'Không rõ')})"
  ```



Cấu trúc này đã tốt, nhưng tôi đề xuất một số điều chỉnh để tối ưu hơn:

```json
{
    "id": "doc_1",
    "vector": [...],
    "payload": {
        "page_content": "Luật này quy định về quy tắc giao thông đường bộ...",
        "metadata": {
            "source": "LegalRAG.xlsx",
            "question": "Trình bày khái niệm, chế độ pháp lý vùng nội thủy...",
            "chapter": "Chương 1",           // Thêm thông tin phân loại
            "category": "Luật Giao thông",   // Thêm danh mục
            "last_updated": "2024-01-01"     // Thêm thông tin version
        }
    }
}
```

Lý do đề xuất:

1. **Cấu trúc hiện tại đã tốt vì**:
   - Tách biệt rõ ràng giữa nội dung (`page_content`) và metadata
   - Có thông tin nguồn (`source`)
   - Có câu hỏi liên quan (`question`)

2. **Có thể cải thiện bằng cách**:
   - Thêm các trường metadata hữu ích
   - Chuẩn hóa format câu hỏi
   - Thêm các trường để dễ dàng tìm kiếm và lọc

3. **Đề xuất format câu hỏi**:
```python
{
    "payload": {
        "page_content": "Nội dung chi tiết...",
        "metadata": {
            "questions": [  // Mảng các câu hỏi thay vì một câu
                {
                    "text": "Trình bày khái niệm, chế độ pháp lý vùng nội thủy?",
                    "type": "definition",  // Loại câu hỏi
                    "keywords": ["khái niệm", "pháp lý", "nội thủy"]
                }
            ],
            "source": {
                "file": "LegalRAG.xlsx",
                "sheet": "Sheet1",
                "row": 1
            }
        }
    }
}
```

4. **Code xử lý sẽ cần điều chỉnh**:
```python:pipelines/qdant_pipeline_zohaibMeExtending.py
def pipe(self, user_message: str, model_id: str, messages: List[dict], body: dict) -> Union[str, Generator, Iterator]:
    try:
        # Tìm kiếm câu hỏi trùng khớp
        exact_match = self.qdrant_client.scroll(
            collection_name=self.valves.QDRANT_COLLECTION,
            scroll_filter=models.Filter(
                must=[
                    models.FieldCondition(
                        key="metadata.questions.text",  # Đường dẫn mới
                        match=models.MatchValue(value=user_message.lower().strip())
                    )
                ]
            ),
            limit=1
        )
        
        if exact_match and len(exact_match[0]) > 0:
            match = exact_match[0][0]
            return {
                "answer": match.payload.get('page_content', ''),
                "source": match.payload.get('metadata', {}).get('source', {}).get('file', ''),
                "category": match.payload.get('metadata', {}).get('category', '')
            }
            
        # Tiếp tục xử lý vector search nếu không tìm thấy
        # ...

    except Exception as e:
        print(f"Pipeline error: {str(e)}")
        return f"Error: {str(e)}"
```

5. **Lợi ích của cấu trúc mới**:
   - Dễ dàng mở rộng thêm thông tin metadata
   - Hỗ trợ nhiều câu hỏi cho một nội dung
   - Dễ dàng tìm kiếm và lọc theo nhiều tiêu chí
   - Có thể thêm thông tin phiên bản và cập nhật
   - Cấu trúc nguồn chi tiết hơn

6. **Lưu ý khi triển khai**:
   - Cần cập nhật code indexing để hỗ trợ cấu trúc mới
   - Đảm bảo các trường metadata được index đúng cách
   - Có thể cần migration data nếu đã có dữ liệu cũ

Bạn nghĩ sao về những đề xuất này?
